<a href="https://colab.research.google.com/github/pratim808/TTS_Bengali/blob/main/Copy_of_Fine_Tune_SpeechT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets soundfile accelerate speechbrain==0.5.16

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 44.4 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import load_dataset, Audio
#https://huggingface.co/datasets/erenfazlioglu/turkishvoicedataset
dataset = load_dataset("arif11/Bengali_AI_Speech", split="train")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/450 [00:00<?, ?B/s]

(…)-00000-of-00002-a03a79119f68cd6c.parquet:   0%|          | 0.00/428M [00:00<?, ?B/s]

(…)-00001-of-00002-701cda474601397f.parquet:   0%|          | 0.00/430M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32762 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'transcription', 'split'],
    num_rows: 32762
})

In [5]:
dataset = dataset.remove_columns("split")

In [6]:
dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 32762
})

In [7]:
len(dataset)

32762

In [8]:
print(dataset)

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 32762
})


In [9]:
# Calculate the number of rows for half the dataset
half_size = len(dataset) // 2

# Select the first half of the dataset
dataset = dataset.select(range(half_size))

print(dataset)

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 16381
})


We are using just the 1/16th of the data

In [10]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [11]:
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
tokenizer = processor.tokenizer

In [13]:
dataset[2:5]

{'audio': [{'path': '00001e0bc131.mp3',
   'array': array([ 2.27373675e-13,  3.41060513e-13, -2.27373675e-13, ...,
          -1.78905611e-06,  3.38623067e-08,  3.59068508e-07]),
   'sampling_rate': 16000},
  {'path': '000024b3d810.mp3',
   'array': array([ 6.93889390e-18, -6.93889390e-18, -1.04083409e-17, ...,
          -4.54747351e-13, -1.13686838e-13, -7.95807864e-13]),
   'sampling_rate': 16000},
  {'path': '000028220ab3.mp3',
   'array': array([ 0.00000000e+00,  2.91038305e-11, -2.32830644e-10, ...,
           2.77555756e-17, -1.38777878e-17,  2.77555756e-17]),
   'sampling_rate': 16000}],
 'transcription': ['আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।',
  'নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন করেন।',
  'হুমম, ওহ হেই, দেখো।']}

Let's normalize the dataset, create a column called "normalized_text"

In [14]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/16381 [00:00<?, ? examples/s]

In [15]:
dataset_vocab - tokenizer_vocab

{' ',
 '।',
 'ঁ',
 'ং',
 'ঃ',
 'অ',
 'আ',
 'ই',
 'ঈ',
 'উ',
 'ঊ',
 'ঋ',
 'এ',
 'ঐ',
 'ও',
 'ঔ',
 'ক',
 'খ',
 'গ',
 'ঘ',
 'ঙ',
 'চ',
 'ছ',
 'জ',
 'ঝ',
 'ঞ',
 'ট',
 'ঠ',
 'ড',
 'ঢ',
 'ণ',
 'ত',
 'থ',
 'দ',
 'ধ',
 'ন',
 'প',
 'ফ',
 'ব',
 'ভ',
 'ম',
 'য',
 'র',
 'ল',
 'শ',
 'ষ',
 'স',
 'হ',
 '়',
 'া',
 'ি',
 'ী',
 'ু',
 'ূ',
 'ৃ',
 'ে',
 'ৈ',
 'ো',
 'ৌ',
 '্',
 'ৎ',
 'ড়',
 'ঢ়',
 'য়',
 'ৰ',
 '৷',
 '–',
 '‘',
 '’'}

In [16]:
import re

def normalize_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove specific signs
    text = re.sub(r"[৷–‘’]", "", text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Define a function to add the normalized_text column
def add_normalized_text(example):
    example['normalized_text'] = normalize_text(example['transcription'])
    return example

# Apply the function to the dataset
dataset = dataset.map(add_normalized_text)

# Print the first few examples to verify
print(dataset[2:5])

Map:   0%|          | 0/16381 [00:00<?, ? examples/s]

{'audio': [{'path': '00001e0bc131.mp3', 'array': array([ 2.27373675e-13,  3.41060513e-13, -2.27373675e-13, ...,
       -1.78905611e-06,  3.38623067e-08,  3.59068508e-07]), 'sampling_rate': 16000}, {'path': '000024b3d810.mp3', 'array': array([ 6.93889390e-18, -6.93889390e-18, -1.04083409e-17, ...,
       -4.54747351e-13, -1.13686838e-13, -7.95807864e-13]), 'sampling_rate': 16000}, {'path': '000028220ab3.mp3', 'array': array([ 0.00000000e+00,  2.91038305e-11, -2.32830644e-10, ...,
        2.77555756e-17, -1.38777878e-17,  2.77555756e-17]), 'sampling_rate': 16000}], 'transcription': ['আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।', 'নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন করেন।', 'হুমম, ওহ হেই, দেখো।'], 'normalized_text': ['আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।', 'নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন করেন।', 'হুমম, ওহ হেই, দেখো।']}


In [17]:
import re

def normalize_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation (except apostrophes)
    text = re.sub(r'[^\w\s\']', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Define a function to add the normalized_text column
def add_normalized_text(example):
    example['normalized_text'] = normalize_text(example['transcription'])
    return example

# Apply the function to the dataset
dataset = dataset.map(add_normalized_text)

# Print the first few examples to verify
print(dataset[2:5])

Map:   0%|          | 0/16381 [00:00<?, ? examples/s]

{'audio': [{'path': '00001e0bc131.mp3', 'array': array([ 2.27373675e-13,  3.41060513e-13, -2.27373675e-13, ...,
       -1.78905611e-06,  3.38623067e-08,  3.59068508e-07]), 'sampling_rate': 16000}, {'path': '000024b3d810.mp3', 'array': array([ 6.93889390e-18, -6.93889390e-18, -1.04083409e-17, ...,
       -4.54747351e-13, -1.13686838e-13, -7.95807864e-13]), 'sampling_rate': 16000}, {'path': '000028220ab3.mp3', 'array': array([ 0.00000000e+00,  2.91038305e-11, -2.32830644e-10, ...,
        2.77555756e-17, -1.38777878e-17,  2.77555756e-17]), 'sampling_rate': 16000}], 'transcription': ['আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।', 'নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন করেন।', 'হুমম, ওহ হেই, দেখো।'], 'normalized_text': ['আম তমর কষটট বঝছ কনত এট সঠক পথ ন', 'নচ শষ হওযর পর সকল শরর ধয একসঙগ ভজন করন', 'হমম ওহ হই দখ']}


In [18]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/16381 [00:00<?, ? examples/s]

In [19]:
dataset_vocab - tokenizer_vocab

{' ',
 'অ',
 'আ',
 'ই',
 'ঈ',
 'উ',
 'ঊ',
 'ঋ',
 'এ',
 'ঐ',
 'ও',
 'ঔ',
 'ক',
 'খ',
 'গ',
 'ঘ',
 'ঙ',
 'চ',
 'ছ',
 'জ',
 'ঝ',
 'ঞ',
 'ট',
 'ঠ',
 'ড',
 'ঢ',
 'ণ',
 'ত',
 'থ',
 'দ',
 'ধ',
 'ন',
 'প',
 'ফ',
 'ব',
 'ভ',
 'ম',
 'য',
 'র',
 'ল',
 'শ',
 'ষ',
 'স',
 'হ',
 'ৎ',
 'ড়',
 'ঢ়',
 'য়',
 'ৰ',
 '৷'}

In [20]:
replacements = [


    ('া', 'aa'),  # Long a
    ('ৌ', 'ou'),  # Ch as in "chair"
    ('ি', 'ei'),  # Silent g or slight elongation of the preceding vowel
    ('ী', 'ei'),   # Dotless i
    ('ে', 'ea'),   # Long i
    ('ৈ', 'oi'),  # Similar to German ö
    ('ো', 'o'),  # Sh as in "shoe"
    ('ু', 'eu'),  # Similar to German ü
    ('ূ', 'eu'),
    ( 'ৃ','ri'),
    ('়','ro'),
    ('অ','A'),
    ('আ','A'),
    ('ই','I'),
    ('ঈ','I'),
    ('উ','U'),
    ('ঊ','U'),
    ('ঋ','R'),
    ('এ','E'),
    ('ঐ','E'),
    ('ও','O'),
    ('ঔ','O'),
    ('ক','k'),
    ('খ','kh'),
    ('গ','g'),
    ('ঘ','gh'),
    ('ঙ','ng'),
    ('চ','ch'),
    ('ছ','chh'),
    ('জ','j'),
    ('ঝ','j'),
    ('ঞ','nj'),
    ('ট','t'),
    ('ঠ','th'),
    ('ড','d'),
    ('ঢ','dh'),
    ('ণ','n'),
    ('ত','t'),
    ('থ','th'),
    ('দ','d'),
    ('ধ','dh'),
    ('ন','n'),
    ('প','p'),
    ('ফ','f'),
    ('ব','b'),
    ('ভ','v'),
    ('ম','m'),
    ('য','y'),
    ('র','r'),
    ('ল','l'),
    ('শ','sh'),
    ('ষ','s'),
    ('স','s'),
    ('হ','h'),
    ('ড়','r'),
    ('ঢ়','r'),
    ('য়','y'),
    ('ৎ','t'),
    ('ং','ng'),
    ('ঃ','h'),
    ('ঁ','n'),



]

def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["transcription"] = inputs["transcription"].replace(src, dst)
    return inputs

dataset = dataset.map(cleanup_text)

Map:   0%|          | 0/16381 [00:00<?, ? examples/s]

In [21]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:147: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(path, map_location=device), strict=Fal

In [22]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [23]:
processed_example = prepare_dataset(dataset[0])
list(processed_example.keys())

['input_ids', 'labels', 'speaker_embeddings']

In [24]:
processed_example["speaker_embeddings"].shape

(512,)

In [25]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

Map:   0%|          | 0/16381 [00:00<?, ? examples/s]

In [26]:
dataset

Dataset({
    features: ['input_ids', 'labels', 'speaker_embeddings'],
    num_rows: 16381
})

In [27]:
dataset[2:5]

{'input_ids': [[4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 2],
  [4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 2],
  [4, 3, 4, 3, 4, 3, 4, 3, 2]],
 'labels': [[[-7.607634544372559,
    -7.628145694732666,
    -7.795763969421387,
    -7.998585224151611,
    -7.597989559173584,
    -7.707141399383545,
    -7.556214332580566,
    -7.444979667663574,
    -7.630274772644043,
    -7.440500736236572,
    -7.563518047332764,
    -7.437647819519043,
    -7.252604961395264,
    -7.572073936462402,
    -7.294164180755615,
    -7.2629804611206055,
    -7.731696128845215,
    -7.344573974609375,
    -7.49266242980957,
    -7.783534049987793,
    -7.5154523849487305,
    -7.675900936126709,
    -7.6940741539001465,
    -7.6009392738342285,
    -7.492278099060059,
    -7.3092522621154785,
    -7.727921962738037,
    -7.546874523162842,
    -7.542690753936768,
    -7.799766540527344,
    -7.754504680633545,
    -7.767696857452393,
    -7.429049491882324,
    -7.61258316040039

In [28]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

Filter:   0%|          | 0/16381 [00:00<?, ? examples/s]

16381

In [29]:
dataset = dataset.train_test_split(test_size=0.1)

In [30]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [31]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [32]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

In [33]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [37]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="speecht5_finetuned_bengali_tr",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [38]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [39]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
1000,0.702300,0.639473
2000,0.663100,0.620599
3000,0.655900,0.606098
4000,0.646500,0.599261
5000,0.631200,0.598246


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1876}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/e

TrainOutput(global_step=5000, training_loss=0.6757854959487916, metrics={'train_runtime': 5461.1941, 'train_samples_per_second': 29.298, 'train_steps_per_second': 0.916, 'total_flos': 4137015665144736.0, 'train_loss': 0.6757854959487916, 'epoch': 10.845986984815617})

In [ ]:
trainer.push_to_hub()

# Inference

In [40]:
model = SpeechT5ForTextToSpeech.from_pretrained(
    "/content/speecht5_finetuned_bengali_tr"
)

In [41]:
example = dataset["test"][304]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

In [49]:
text = "কেমন আছো, আমি তোমাকে ভালবাসি!"

In [50]:
import re

number_words = {
    0: "শূন্য", 1: "এক", 2: "দুই", 3: "তিন", 4: "চার", 5: "পাঁচ", 6: "ছয়", 7: "সাত", 8: "আট", 9: "নয়",
    10: "দশ", 11: "এগারো", 12: "বারো", 13: "তেরো", 14: "চৌদ্দ", 15: "পনেরো", 16: "ষোল", 17: "সতেরো",
    18: "আঠারো", 19: "ঊনিশ", 20: "বিশ", 30: "ত্রিশ", 40: "চল্লিশ", 50: "পঞ্চাশ", 60: "ষাট", 70: "সত্তর",
    80: "আশি", 90: "নব্বই", 100: "একশ", 1000: "এক হাজার"
}

def number_to_words(number):
    if number < 20:
        return number_words[number]
    elif number < 100:
        tens, unit = divmod(number, 10)
        return number_words[tens * 10] + (" " + number_words[unit] if unit else "")
    elif number < 1000:
        hundreds, remainder = divmod(number, 100)
        return (number_words[hundreds] + " শত" if hundreds > 1 else "একশ") + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000:
        thousands, remainder = divmod(number, 1000)
        return (number_to_words(thousands) + " হাজার" if thousands > 1 else "এক হাজার") + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000:
        millions, remainder = divmod(number, 1000000)
        return number_to_words(millions) + " মিলিয়ন" + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000000:
        billions, remainder = divmod(number, 1000000000)
        return number_to_words(billions) + " বিলিয়ন" + (" " + number_to_words(remainder) if remainder else "")
    else:
        return str(number)

def replace_numbers_with_words(text):

    def replace(match):
        number = int(match.group())
        return number_to_words(number)

    # Find the numbers and change with words.
    result = re.sub(r'\b\d+\b', replace, text)

    return result

In [ ]:
number_words = {
    0: "sıfır", 1: "bir", 2: "iki", 3: "üç", 4: "dört", 5: "beş", 6: "altı", 7: "yedi", 8: "sekiz", 9: "dokuz",
    10: "on", 11: "on bir", 12: "on iki", 13: "on üç", 14: "on dört", 15: "on beş", 16: "on altı", 17: "on yedi",
    18: "on sekiz", 19: "on dokuz", 20: "yirmi", 30: "otuz", 40: "kırk", 50: "elli", 60: "altmış", 70: "yetmiş",
    80: "seksen", 90: "doksan", 100: "yüz", 1000: "bin"
}

def number_to_words(number):
    if number < 20:
        return number_words[number]
    elif number < 100:
        tens, unit = divmod(number, 10)
        return number_words[tens * 10] + (" " + number_words[unit] if unit else "")
    elif number < 1000:
        hundreds, remainder = divmod(number, 100)
        return (number_words[hundreds] + " yüz" if hundreds > 1 else "yüz") + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000:
        thousands, remainder = divmod(number, 1000)
        return (number_to_words(thousands) + " bin" if thousands > 1 else "bin") + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000:
        millions, remainder = divmod(number, 1000000)
        return number_to_words(millions) + " milyon" + (" " + number_to_words(remainder) if remainder else "")
    elif number < 1000000000000:
        billions, remainder = divmod(number, 1000000000)
        return number_to_words(billions) + " milyar" + (" " + number_to_words(remainder) if remainder else "")
    else:
        return str(number)

def replace_numbers_with_words(text):

    def replace(match):
        number = int(match.group())
        return number_to_words(number)

    # Find the numbers and change with words.
    result = re.sub(r'\b\d+\b', replace, text)

    return result

In [51]:
# Function to clean up text using the replacement pairs
def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [52]:
converted_text = replace_numbers_with_words(text)
cleaned_text = cleanup_text(converted_text)
final_text = normalize_text(cleaned_text)
final_text

'keamn achho amei tomaakea vaalbaasei'

In [53]:
inputs = processor(text=final_text, return_tensors="pt")

In [54]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [56]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)
# Save the audio to a file (e.g., 'output.wav')
sf.write('output_1.wav', speech.numpy(), 16000)